In [1]:
%matplotlib inline 
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import tmdbsimple as tmdb
from imdb import IMDb
import numpy as np
import pandas as pd
import time
import cPickle as pickle
import io

In [53]:
MovieTable = pd.read_pickle('MovieData_10_15.pkl')
MovieTable = pd.DataFrame(MovieTable).transpose()
MovieTable.head()

,budget,cast,director,genres,gross,id,imdb_id,imdb_rating,mpaa,name,opening_gross,opening_theathers,runtime,year
189,65000000,"[0000620, 0004695, 0000982, 0330687, 0206257, ...","[0588340, 0001675]","[Action, Crime, Thriller]",13757804,189,tt0458481,6.5,Rated R for strong brutal stylized violence th...,Sin City: A Dame to Kill For,6317683,2894,102,2014
1865,380000000,"[0000136, 0004851, 0001691, 0574534, 0573618, ...",[0551128],"[Action, Adventure, Fantasy]",241071802,1865,tt1298650,6.7,"Rated PG-13 for sexual content/nudity, languag...",Pirates of the Caribbean: On Stranger Tides,90151958,4155,136,2011
1930,215000000,"[1940449, 1297015, 0406975, 0001459, 0000640, ...",[1989536],"[Action, Adventure]",262030663,1930,tt0948470,7,Rated PG-13 for sequences of action and violence,The Amazing Spider-Man,62004688,4318,136,2012
4258,20000000,"[0864308, 0005350, 3856188, 4795991, 5777210, ...","[0002700, 0001878]",[Comedy],32015787,4258,tt0795461,3.5,Rated PG-13 for crude and sexual content throu...,Scary Movie 5,14157367,3402,86,2013
7978,150000000,"[0581163, 2247629, 1289434, 0001125, 3785649, ...",[0002653],"[Drama, Fantasy, Horror, Thriller]",61979680,7978,tt0780653,5.8,Rated R for bloody horror violence and gore,The Wolfman,31479235,3222,102,2010


In [3]:
MovieTable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2390 entries, 189 to 460654
Data columns (total 14 columns):
budget               2390 non-null object
cast                 2197 non-null object
director             2237 non-null object
genres               2250 non-null object
gross                2390 non-null object
id                   2390 non-null object
imdb_id              2390 non-null object
imdb_rating          2201 non-null object
mpaa                 1297 non-null object
name                 2390 non-null object
opening_gross        2257 non-null object
opening_theathers    2257 non-null object
runtime              2249 non-null object
year                 2257 non-null object
dtypes: object(14)
memory usage: 280.1+ KB


In [54]:
# remove NaN
MovieTable = MovieTable[pd.notnull(MovieTable['imdb_rating'])]
MovieTable = MovieTable[pd.notnull(MovieTable['cast'])]
MovieTable = MovieTable[pd.notnull(MovieTable['director'])]
MovieTable = MovieTable[pd.notnull(MovieTable['genres'])]

In [5]:
MovieTable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2139 entries, 189 to 458877
Data columns (total 14 columns):
budget               2139 non-null object
cast                 2139 non-null object
director             2139 non-null object
genres               2139 non-null object
gross                2139 non-null object
id                   2139 non-null object
imdb_id              2139 non-null object
imdb_rating          2139 non-null object
mpaa                 1295 non-null object
name                 2139 non-null object
opening_gross        2139 non-null object
opening_theathers    2139 non-null object
runtime              2135 non-null object
year                 2139 non-null object
dtypes: object(14)
memory usage: 250.7+ KB


In [55]:
# shorter mpaa
for movie in range(len(MovieTable)):
    if MovieTable['mpaa'].iloc[movie] != None:
        if MovieTable['mpaa'].iloc[movie].split()[1] in ['R', 'PG', 'PG-13', 'NC-17']:
            MovieTable['mpaa'].iloc[movie] = MovieTable['mpaa'].iloc[movie].split()[1]
        else:
            MovieTable['mpaa'].iloc[movie] = 'R'

# fill none mpaa with mode value
MovieTable['mpaa'].fillna(MovieTable['mpaa'].mode()[0], inplace=True)

In [47]:
MovieTable['mpaa'].value_counts()

R        295
PG-13     86
PG        39
Name: mpaa, dtype: int64

In [8]:
df_with_dummies = pd.get_dummies(MovieTable, columns = ['mpaa'])

In [9]:
df_with_dummies.head()

,budget,cast,director,genres,gross,id,imdb_id,imdb_rating,name,opening_gross,opening_theathers,runtime,year,mpaa_NC-17,mpaa_PG,mpaa_PG-13,mpaa_R
189,65000000,"[0000620, 0004695, 0000982, 0330687, 0206257, ...","[0588340, 0001675]","[Action, Crime, Thriller]",13757804,189,tt0458481,6.5,Sin City: A Dame to Kill For,6317683,2894,102,2014,0,0,0,1
1865,380000000,"[0000136, 0004851, 0001691, 0574534, 0573618, ...",[0551128],"[Action, Adventure, Fantasy]",241071802,1865,tt1298650,6.7,Pirates of the Caribbean: On Stranger Tides,90151958,4155,136,2011,0,0,1,0
1930,215000000,"[1940449, 1297015, 0406975, 0001459, 0000640, ...",[1989536],"[Action, Adventure]",262030663,1930,tt0948470,7,The Amazing Spider-Man,62004688,4318,136,2012,0,0,1,0
4258,20000000,"[0864308, 0005350, 3856188, 4795991, 5777210, ...","[0002700, 0001878]",[Comedy],32015787,4258,tt0795461,3.5,Scary Movie 5,14157367,3402,86,2013,0,0,1,0
7978,150000000,"[0581163, 2247629, 1289434, 0001125, 3785649, ...",[0002653],"[Drama, Fantasy, Horror, Thriller]",61979680,7978,tt0780653,5.8,The Wolfman,31479235,3222,102,2010,0,0,0,1


In [10]:
MovieTable.head()

,budget,cast,director,genres,gross,id,imdb_id,imdb_rating,mpaa,name,opening_gross,opening_theathers,runtime,year
189,65000000,"[0000620, 0004695, 0000982, 0330687, 0206257, ...","[0588340, 0001675]","[Action, Crime, Thriller]",13757804,189,tt0458481,6.5,R,Sin City: A Dame to Kill For,6317683,2894,102,2014
1865,380000000,"[0000136, 0004851, 0001691, 0574534, 0573618, ...",[0551128],"[Action, Adventure, Fantasy]",241071802,1865,tt1298650,6.7,PG-13,Pirates of the Caribbean: On Stranger Tides,90151958,4155,136,2011
1930,215000000,"[1940449, 1297015, 0406975, 0001459, 0000640, ...",[1989536],"[Action, Adventure]",262030663,1930,tt0948470,7,PG-13,The Amazing Spider-Man,62004688,4318,136,2012
4258,20000000,"[0864308, 0005350, 3856188, 4795991, 5777210, ...","[0002700, 0001878]",[Comedy],32015787,4258,tt0795461,3.5,PG-13,Scary Movie 5,14157367,3402,86,2013
7978,150000000,"[0581163, 2247629, 1289434, 0001125, 3785649, ...",[0002653],"[Drama, Fantasy, Horror, Thriller]",61979680,7978,tt0780653,5.8,R,The Wolfman,31479235,3222,102,2010


### Create Actor table

In [11]:
actorDict = dict()
for movie in range(len(MovieTable)):
    for actor in MovieTable['cast'].iloc[movie]:
        if actor not in actorDict.keys():
            actorDict[actor] = dict()
            actorDict[actor]['imdb_rating'] = list()
            actorDict[actor]['budget'] = list()
            actorDict[actor]['gross'] = list()
            actorDict[actor]['movie_id'] = list()
        actorDict[actor]['imdb_rating'].append(MovieTable['imdb_rating'].iloc[movie])
        actorDict[actor]['budget'].append(MovieTable['budget'].iloc[movie])
        actorDict[actor]['gross'].append(MovieTable['gross'].iloc[movie])
        actorDict[actor]['movie_id'].append(MovieTable['id'].iloc[movie])

In [12]:
for actor in actorDict.keys():
    actorDict[actor]['movie_count'] = len(actorDict[actor]['movie_id'])
    actorDict[actor]['gross_sum'] = sum(actorDict[actor]['gross'])
    actorDict[actor]['gross_avg'] = actorDict[actor]['gross_sum']/float(actorDict[actor]['movie_count'])
    actorDict[actor]['imdb_rating_sum'] = sum(actorDict[actor]['imdb_rating'])
    actorDict[actor]['imdb_rating_avg'] = actorDict[actor]['imdb_rating_sum']/float(actorDict[actor]['movie_count'])

In [13]:
# for actor in actorDict.keys():
#     if actorDict[actor]['movie_count'] < 5:
#         actorDict.pop(actor, None)

In [14]:
pickle.dump(actorDict, io.open('ActorTable_10_15.pkl', 'wb'))

In [15]:
ActorT = pd.read_pickle('ActorTable_10_15.pkl')
ActorDF = pd.DataFrame(ActorT).transpose()

In [16]:
MDdf.head()

NameError: name 'MDdf' is not defined

Creat director table

In [17]:
directorDict = dict()
for movie in range(len(MovieTable)):
    for director in MovieTable['director'].iloc[movie]:
        if director not in directorDict.keys():
            directorDict[director] = dict()
            directorDict[director]['imdb_rating'] = list()
            directorDict[director]['budget'] = list()
            directorDict[director]['gross'] = list()
            directorDict[director]['movie_id'] = list()
        directorDict[director]['imdb_rating'].append(MovieTable['imdb_rating'].iloc[movie])
        directorDict[director]['budget'].append(MovieTable['budget'].iloc[movie])
        directorDict[director]['gross'].append(MovieTable['gross'].iloc[movie])
        directorDict[director]['movie_id'].append(MovieTable['id'].iloc[movie])

In [18]:
for director in directorDict.keys():
    directorDict[director]['movie_count'] = len(directorDict[director]['movie_id'])
    directorDict[director]['gross_sum'] = sum(directorDict[director]['gross'])
    directorDict[director]['gross_avg'] = directorDict[director]['gross_sum']/float(directorDict[director]['movie_count'])
    directorDict[director]['imdb_rating_sum'] = sum(directorDict[director]['imdb_rating'])
    directorDict[director]['imdb_rating_avg'] = directorDict[director]['imdb_rating_sum']/float(directorDict[director]['movie_count'])

In [19]:
pickle.dump(directorDict, io.open('DirectorTable_10_15.pkl', 'wb'))


In [20]:
DirectorT = pd.read_pickle('DirectorTable_10_15.pkl')
DirectorDF = pd.DataFrame(DirectorT).transpose()

In [21]:
DirectorDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2142 entries, 0000095 to 8377173
Data columns (total 9 columns):
budget             2142 non-null object
gross              2142 non-null object
gross_avg          2142 non-null object
gross_sum          2142 non-null object
imdb_rating        2142 non-null object
imdb_rating_avg    2142 non-null object
imdb_rating_sum    2142 non-null object
movie_count        2142 non-null object
movie_id           2142 non-null object
dtypes: object(9)
memory usage: 167.3+ KB


In [22]:
MDdf.head()

NameError: name 'MDdf' is not defined

genres table

In [23]:
MovieTable.genres.value_counts()
genres_list = list()
for gl in MovieTable.genres.get_values():
    if gl == None:
        continue
    for g in gl:
        if g not in genres_list:
            genres_list.append(g)


In [24]:
genreDict = dict()
for movie in range(len(MovieTable)):
    if not MovieTable['genres'].iloc[movie]:
        print movie
        continue
    for genre in MovieTable['genres'].iloc[movie]:
        if genre not in genreDict.keys():
            genreDict[genre] = dict()
            genreDict[genre]['imdb_rating'] = list()
            genreDict[genre]['budget'] = list()
            genreDict[genre]['gross'] = list()
            genreDict[genre]['movie_id'] = list()
        genreDict[genre]['imdb_rating'].append(MovieTable['imdb_rating'].iloc[movie])
        genreDict[genre]['budget'].append(MovieTable['budget'].iloc[movie])
        genreDict[genre]['gross'].append(MovieTable['gross'].iloc[movie])
        genreDict[genre]['movie_id'].append(MovieTable['id'].iloc[movie])

In [25]:
for genre in genreDict.keys():
    genreDict[genre]['movie_count'] = len(genreDict[genre]['movie_id'])
    genreDict[genre]['gross_sum'] = sum(genreDict[genre]['gross'])
    genreDict[genre]['gross_avg'] = genreDict[genre]['gross_sum']/float(genreDict[genre]['movie_count'])
    genreDict[genre]['imdb_rating_sum'] = sum(genreDict[genre]['imdb_rating'])
    genreDict[genre]['imdb_rating_avg'] = genreDict[genre]['imdb_rating_sum']/float(genreDict[genre]['movie_count'])

In [26]:
pickle.dump(genreDict, io.open('GenresTable_10_15.pkl', 'wb'))


In [27]:
GenresT = pd.read_pickle('GenresTable_10_15.pkl')
GenresDF = pd.DataFrame(GenresT).transpose()
GenresDF.head()

,budget,gross,gross_avg,gross_sum,imdb_rating,imdb_rating_avg,imdb_rating_sum,movie_count,movie_id
Action,"[65000000, 380000000, 215000000, 150000000, 15...","[13757804, 241071802, 262030663, 90759676, 131...",4.96474e+07,18568138885,"[6.5, 6.7, 7.0, 6.6, 4.2, 7.4, 5.8, 6.9, 6.8, ...",6.10535,2283.4,374,"[189, 1865, 1930, 9543, 10196, 17578, 18823, 2..."
Adventure,"[380000000, 215000000, 150000000, 155000000, 1...","[241071802, 262030663, 90759676, 104386950, 23...",8.23964e+07,19692730128,"[6.7, 7.0, 6.6, 6.3, 6.3, 8.3, 4.2, 7.7, 8.1, ...",6.45732,1543.3,239,"[1865, 1930, 9543, 10140, 10192, 10193, 10196,..."
Animation,"[165000000, 200000000, 20000000, 130000000, 13...","[238736787, 415004880, 25107267, 77591831, 148...",7.86131e+07,7311021753,"[6.3, 8.3, 5.3, 7.4, 7.3, 7.5, 7.0, 4.6, 7.4, ...",6.59032,612.9,93,"[10192, 10193, 12819, 17578, 38055, 41201, 412..."
Biography,"[10000000, 40000000, 22000000, 35000000, 0, 0,...","[3573673, 96962694, 9540691, 59713955, 137885,...",1.40573e+07,2937977406,"[6.6, 7.7, 6.8, 7.2, 5.6, 7.4, 7.8, 7.1, 7.6, ...",7.0244,1468.1,209,"[27586, 37799, 38363, 39486, 39845, 41397, 439..."
Comedy,"[20000000, 165000000, 200000000, 28000000, 200...","[32015787, 238736787, 415004880, 7002261, 2510...",2.80866e+07,19070802271,"[3.5, 6.3, 8.3, 6.1, 5.3, 5.5, 7.5, 5.0, 6.4, ...",6.06392,4117.4,679,"[4258, 10192, 10193, 10317, 12819, 13477, 2253..."


In [28]:
ActorDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73641 entries, 0000002 to 9089683
Data columns (total 9 columns):
budget             73641 non-null object
gross              73641 non-null object
gross_avg          73641 non-null object
gross_sum          73641 non-null object
imdb_rating        73641 non-null object
imdb_rating_avg    73641 non-null object
imdb_rating_sum    73641 non-null object
movie_count        73641 non-null object
movie_id           73641 non-null object
dtypes: object(9)
memory usage: 5.6+ MB


In [56]:
MovieTable['director_boxoffice'] = 0
MovieTable['cast_boxoffice'] = 0
MovieTable['genres_boxoffice'] = 0

In [57]:
for movie in range(len(MovieTable)):
    ld = len(MovieTable['director'].iloc[movie])
    for director in MovieTable['director'].iloc[movie]:
        try:
            MovieTable.director_boxoffice.iat[movie] += DirectorDF.gross_avg.at[director]
            
        except:
            ld -= 1
            print 'director', director
    if ld>0:
        MovieTable.director_boxoffice.iat[movie] /= float(ld)
    
    ld = len(MovieTable['cast'].iloc[movie])
    for actor in MovieTable['cast'].iloc[movie]:
        try:
            MovieTable.cast_boxoffice.iat[movie] += ActorDF.gross_avg.at[actor]
            
        except:
            ld -= 1
            print 'actor', actor
    if ld>0:
        MovieTable.cast_boxoffice.iat[movie] /= float(ld)
    
    
    for genre in MovieTable['genres'].iloc[movie]:
        MovieTable.genres_boxoffice.iat[movie] += GenresDF.gross_avg.at[genre]
    MovieTable.genres_boxoffice.iat[movie] /= float(len(MovieTable['genres'].iloc[movie]))

In [58]:
MovieTable.to_pickle('MovieDataClean_10_15.pkl')

In [59]:
MovieTable.head()

,budget,cast,director,genres,gross,id,imdb_id,imdb_rating,mpaa,name,opening_gross,opening_theathers,runtime,year,director_boxoffice,cast_boxoffice,genres_boxoffice
189,65000000,"[0000620, 0004695, 0000982, 0330687, 0206257, ...","[0588340, 0001675]","[Action, Crime, Thriller]",13757804,189,tt0458481,6.5,R,Sin City: A Dame to Kill For,6317683,2894,102,2014,16929594,30220545,36087236
1865,380000000,"[0000136, 0004851, 0001691, 0574534, 0573618, ...",[0551128],"[Action, Adventure, Fantasy]",241071802,1865,tt1298650,6.7,PG-13,Pirates of the Caribbean: On Stranger Tides,90151958,4155,136,2011,184537087,174422411,70050167
1930,215000000,"[1940449, 1297015, 0406975, 0001459, 0000640, ...",[1989536],"[Action, Adventure]",262030663,1930,tt0948470,7,PG-13,The Amazing Spider-Man,62004688,4318,136,2012,232442298,193142424,66021895
4258,20000000,"[0864308, 0005350, 3856188, 4795991, 5777210, ...","[0002700, 0001878]",[Comedy],32015787,4258,tt0795461,3.5,PG-13,Scary Movie 5,14157367,3402,86,2013,41643139,50143497,28086601
7978,150000000,"[0581163, 2247629, 1289434, 0001125, 3785649, ...",[0002653],"[Drama, Fantasy, Horror, Thriller]",61979680,7978,tt0780653,5.8,R,The Wolfman,31479235,3222,102,2010,61979680,70730664,36944285
